In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import wordnet
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import os

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM

from keras.layers import Bidirectional
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.models import model_from_json
from keras.models import load_model

In [ ]:
df = pd.read_csv("/kaggle/input/515k-hotel-reviews-data-in-europe/Hotel_Reviews.csv") 

In [ ]:
df

In [ ]:
df['Review']=df['Negative_Review']+df['Positive_Review']

In [ ]:
df['good_review']=df["Reviewer_Score"].apply(lambda x:1 if x>=5 else 0) #X=1 good rev
                                                                         #X=1 Bad rev

In [ ]:
df_rev=df[['Review','good_review']]

In [ ]:
df_rev

In [ ]:
df_rev.dropna()

In [ ]:
class_count_1, class_count_0 =df_rev['good_review']. value_counts ()
df_rev_0 = df_rev[df_rev['good_review']==0]
df_rev_1=df_rev[df_rev['good_review']==1]
print(class_count_1)
print(class_count_0)

In [ ]:
df_rev_1_under = df_rev_1.sample (class_count_0)

In [ ]:
df_rev_1_under

In [ ]:
dataRev = pd.concat ([df_rev_1_under, df_rev_0], axis = 0)

In [ ]:
labels = dataRev["good_review"]

In [ ]:
def get_wordnet(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
def clean_rev(rev):
    # lower text
    rev = rev.lower()
    rev1=rev.split(" ")
    # tokenize text and remove puncutation
    rev = [word.strip(string.punctuation) for word in rev1]
    # remove words that contain numbers
    rev = [word for word in rev if not any(c.isdigit() for c in word)]
    # remove stop words
    stop_W = stopwords.words('english')
    rev = [word for word in rev if word not in stop_W]
    # remove empty tokens
    rev = [emp for emp in rev if len(emp) > 0]
    # pos tag text
    pos_tags = pos_tag(rev)
    # lemmatize text
    rev = [WordNetLemmatizer().lemmatize(tag[0], get_wordnet(tag[1])) for tag in pos_tags]
    # remove words with only one letter
    rev = [t for t in rev if len(t) >=1]
    # join all
    rev = " ".join(rev)
    return(rev)

In [ ]:
dataRev['Review'] = dataRev['Review'].transform(lambda x: clean_rev(x))

In [ ]:
X = dataRev.drop('good_review', axis=1)
Y = dataRev[['good_review']]

In [ ]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X['Review'])
tokenizer.word_index #how to get the word to index mapping 
sequences = tokenizer.texts_to_sequences(X['Review'])
maxSize = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=maxSize )
print(data.shape)
data

In [ ]:
target=np.array(Y)
target

In [ ]:
acc_per_fold = []
loss_per_fold = []
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
import numpy as np

fold_no = 1
kf = KFold(n_splits = 5, random_state = 5168, shuffle = False)


for train, test in kf.split(data, target):
    
    model_lstm = Sequential()
    model_lstm.add(Embedding(20000, 100, input_length=50))
    model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model_lstm.add(Dense(1, activation='sigmoid')) 
    model_lstm.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')
    # Fit data to model
    history = model_lstm .fit(data[train],target[train] , epochs=3,batch_size=180,verbose=2)
    # Generate generalization metrics
    scores = model_lstm .evaluate(data[test], target[test], verbose=0)
    print(f'Score for fold {fold_no}: {model_lstm .metrics_names[0]} of {scores[0]}; {model_lstm .metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no = fold_no + 1
    # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print('------------------------------------------------------------------------')
        print('Average scores for all folds:')
        print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
        print(f'> Loss: {np.mean(loss_per_fold)}')
        print('------------------------------------------------------------------------')
model.save('model.h5', history)

print("model is created")



In [ ]:
acc_per_fold = []
loss_per_fold = []
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit
import numpy as np
#num_folds = 10
fold_no = 1
kf = KFold(n_splits = 5, random_state = 5168, shuffle = False)
#oof_train_Decision_Tree= np.zeros(train.shape[0])
#for i,(train_index, val_index) in enumerate(kf.split(train)):
for train, test in kf.split(data, target):
    model = Sequential()
    model.add(Embedding(20000, 100, input_length=50))
    model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(1, activation='sigmoid'))
# Adam Optimiser
    model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model .fit(data[train],target[train] , epochs=3,batch_size=128,verbose=2)
  # Generate generalization metrics
    scores = model.evaluate(data[test], target[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model .metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

  # Increase fold number 
    fold_no = fold_no + 1
  # == Provide average scores ==
    print('------------------------------------------------------------------------')
    print('Score per fold')
    for i in range(0, len(acc_per_fold)):
        
        print('------------------------------------------------------------------------')
        print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
        print('------------------------------------------------------------------------')
        print('Average scores for all folds:')
        print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
        print(f'> Loss: {np.mean(loss_per_fold)}')
        print('------------------------------------------------------------------------')
model.save('modelbi.h5', history)
print("model is created")

  # Pour un modèle avec un seule entrée et deux classes (classification binaire) 

#print(model_lstm.summary())